In [32]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.sparse import csc_matrix
from scipy.sparse import csr_matrix

In [42]:
file  = np.load('user_movie_rating.npy')
user_id = file[:,0]
movie_id = file[:,1]
rating = file[:,2]

similarity_threshold = 0.5
print(len(movie_id))

65225506


In [ ]:
#create a sparse matrix of the user-movie rating, where the movies that recieved a rating get a 1 and the ones that did not get a rating get a 0
#We assume from the assignment that we only care about movies that were rated but not the actual ratings. Thus giving a binary matrix

def create_matrix(file):
    user_id = file[:,0]
    movie_id = file[:,1]
    rating = file[:,2]
    data = np.ones(len(rating))
    user_movie_matrix = csc_matrix((data, (user_id, movie_id)))
    return user_movie_matrix

In [67]:
user_movie_matrix = create_matrix(file)
print(user_movie_matrix)

  (95, 1)	1.0
  (155, 1)	1.0
  (441, 1)	1.0
  (575, 1)	1.0
  (853, 1)	1.0
  (1189, 1)	1.0
  (1215, 1)	1.0
  (1264, 1)	1.0
  (1499, 1)	1.0
  (1717, 1)	1.0
  (1718, 1)	1.0
  (1803, 1)	1.0
  (2181, 1)	1.0
  (2326, 1)	1.0
  (2681, 1)	1.0
  (3400, 1)	1.0
  (3670, 1)	1.0
  (3690, 1)	1.0
  (3881, 1)	1.0
  (4756, 1)	1.0
  (4799, 1)	1.0
  (5268, 1)	1.0
  (5691, 1)	1.0
  (5884, 1)	1.0
  (6742, 1)	1.0
  :	:
  (100304, 17770)	1.0
  (100391, 17770)	1.0
  (100507, 17770)	1.0
  (100560, 17770)	1.0
  (100610, 17770)	1.0
  (100664, 17770)	1.0
  (100666, 17770)	1.0
  (100701, 17770)	1.0
  (100720, 17770)	1.0
  (100731, 17770)	1.0
  (100764, 17770)	1.0
  (101076, 17770)	1.0
  (101298, 17770)	1.0
  (101859, 17770)	1.0
  (101923, 17770)	1.0
  (102017, 17770)	1.0
  (102037, 17770)	1.0
  (102107, 17770)	1.0
  (102256, 17770)	1.0
  (102580, 17770)	1.0
  (103055, 17770)	1.0
  (103108, 17770)	1.0
  (103344, 17770)	1.0
  (103382, 17770)	1.0
  (103442, 17770)	1.0


In [ ]:
#now we will be implementing the minhashing techique without using the hash function but using random permutations of the rows of the matrix
#We will be using 100 permutations


0.0